# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   NULL|     BE|   NULL|    NULL|      1|  NULL|   269|  6|    69| NULL|       1|    NULL|    0.0|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070802| 1963| 1096|   NULL|     US|     TX|    NULL|      1|  NULL|     2|  6|    63| NULL|       0|    NULL|   NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070803| 1963| 1096|   NULL|     US|     IL|    NULL|      1|  NULL|     2|  6|    6

In this below step, I created a smaller table from the patents data that only keeps the patent ID and the state it belongs to. I renamed the patent ID column to P_PATENT and made sure it is stored as an integer. I also renamed POSTATE to just STATE so it’s easier to read. Then I filtered the data so that it only includes U.S. patents (COUNTRY = 'US') where the state field is not empty or null.

The result is a list of U.S. patent IDs with their state codes.

In [8]:
from pyspark.sql import functions as F

patent_state = (
    patents
    .select(
        F.col("PATENT").cast("int").alias("P_PATENT"),
        F.col("COUNTRY"),
        F.col("POSTATE").alias("STATE")
    )
    .filter((F.col("COUNTRY") == "US") & F.col("STATE").isNotNull() & (F.col("STATE") != ""))
    .select("P_PATENT", "STATE")
)
patent_state.show(5)


+--------+-----+
|P_PATENT|STATE|
+--------+-----+
| 3070802|   TX|
| 3070803|   IL|
| 3070804|   OH|
| 3070805|   CA|
| 3070806|   PA|
+--------+-----+
only showing top 5 rows



Next, the patent IDs in the citations data were stored as numbers so they could match properly with the patent information. Then I joined the data to find the state of the patent that was cited, which I labeled as CITED_STATE. Next, I did the same for the patent that was citing, calling it CITING_STATE. In the end, I got a table that shows each citation along with the state of the citing patent and the state of the cited patent. This lets me easily see whether citations are happening within the same state or across different states.

In [9]:
# Here CITING/CITED should be ints for joining
cit = citations.select(
    F.col("CITING").cast("int").alias("CITING"),
    F.col("CITED").cast("int").alias("CITED"),
)

# Join to get Cited state
cited_side = (
    cit.join(patent_state.withColumnRenamed("P_PATENT", "CITED"), on="CITED", how="left")
       .withColumnRenamed("STATE", "CITED_STATE")
)

# Join to get Citing state
both_states = (
    cited_side.join(patent_state.withColumnRenamed("P_PATENT", "CITING"), on="CITING", how="left")
              .withColumnRenamed("STATE", "CITING_STATE")
              .select("CITING", "CITED", "CITING_STATE", "CITED_STATE")
)
both_states.show(5)


+-------+-------+------------+-----------+
| CITING|  CITED|CITING_STATE|CITED_STATE|
+-------+-------+------------+-----------+
|3858242|1515701|          MI|       NULL|
|3858241|3634889|          MA|         OH|
|3858241| 956203|          MA|       NULL|
|3858241|1324234|          MA|       NULL|
|3858241|3398406|          MA|         FL|
+-------+-------+------------+-----------+
only showing top 5 rows



The first part filters the dataset to keep only the records where both the citing patent and the cited patent have valid state information, and both states are the same. This gives us only the “same-state” citation edges.
After that, the data is grouped by the citing patent. For each citing patent, the code counts how many times it cited another patent from the same state. This count is stored in a new column called SAME_STATE.
Finally, the results are sorted in descending order of SAME_STATE so that the patents with the highest number of same-state citations are shown at the top. The top 10 such patents are displayed.

In [10]:
same_state_edges = both_states.filter(
    F.col("CITING_STATE").isNotNull() &
    F.col("CITED_STATE").isNotNull() &
    (F.col("CITING_STATE") == F.col("CITED_STATE"))
)

# If rubric wants unique cited patents per citer, we use countDistinct("CITED") instead of count("*")
same_state_counts = (
    same_state_edges.groupBy("CITING")
    .agg(F.count("*").alias("SAME_STATE"))
)
same_state_counts.orderBy(F.desc("SAME_STATE")).show(10, truncate=False)


+-------+----------+
|CITING |SAME_STATE|
+-------+----------+
|5959466|125       |
|5983822|103       |
|6008204|100       |
|5952345|98        |
|5998655|96        |
|5958954|96        |
|5936426|94        |
|5739256|90        |
|5978329|90        |
|5951547|90        |
+-------+----------+
only showing top 10 rows



First, the patent IDs are converted into integers to make sure they match properly during joins. Then, the dataset is joined with the earlier results (same_state_counts) so each patent gets an extra column showing how many same-state citations it has. For patents that don’t appear in the counts table, the value is filled in with 0 to avoid blanks.
Next, the columns are arranged so that the new SAME_STATE column appears at the very end, keeping the structure neat. Finally, the code filters to only U.S. patents, sorts them by the number of same-state citations (highest first, and in case of ties by patent ID), and shows the top 10 results in full detail.

In [11]:

pat_full = patents.withColumn("PATENT", F.col("PATENT").cast("int"))

# Join counts and append SAME_STATE as the final column
pat_with_counts = (
    pat_full.join(same_state_counts.withColumnRenamed("CITING", "PATENT"), on="PATENT", how="left")
            .withColumn("SAME_STATE", F.coalesce(F.col("SAME_STATE"), F.lit(0)))
)

# Reorder columns so SAME_STATE is last
cols_in_order = [c for c in pat_full.columns] + ["SAME_STATE"]

top10 = (
    pat_with_counts
    .filter(F.col("COUNTRY") == "US")
    .select(*cols_in_order)
    .orderBy(F.desc("SAME_STATE"), F.asc("PATENT"))
    .limit(10)
)

top10.show(10, truncate=False)


+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|PATENT |GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|5959466|1999 |14515|1997   |US     |CA     |5310    |2      |NULL  |326   |4  |46    |159  |0       |1.0     |NULL   |0.6186  |NULL    |4.8868  |0.0455  |0.044   |NULL    |NULL    |125       |
|5983822|1999 |14564|1998   |US     |TX     |569900  |2      |NULL  |114   |5  |55    |200  |0       |0.995   |NULL   |0.7201  |NULL    |12.45   |0.0     |0.0     |NULL    |NULL    |103       |
|6008204|1999 |14606|1998   |U